In [13]:
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json
import numpy as np
import pandas as pd
from shapely.geometry import shape, Point
from ipywidgets import jslink, IntSlider, HTML

from traitlets import link

In [3]:
# generate Reindeer positions
nvalues = 10
lons = 7.48 + np.random.rand(nvalues,1)
lons

lats = 60.58 + np.random.rand(nvalues,1)
pd.DataFrame(np.concatenate((lons,lats),axis=1),  columns=["lon", "lat"]).to_csv("reindeers.csv", index=False)

In [4]:
# Read reindeer positions
reindeers = pd.read_csv("reindeers.csv")
reindeers

,lon,lat
0,8.012972,60.926133
1,8.371339,60.827343
2,8.417918,60.756107
3,7.563795,61.138999
4,8.006617,61.472739
5,8.359575,61.562536
6,7.911859,61.489985
7,7.769689,61.130935
8,7.614290,60.637645
9,7.888996,61.143362


In [5]:
lon, lat = 7.48, 60.58
zoom_start = 6
m = ipyl.Map(
#    layers=(basemap_to_tiles(basemaps.OpenTopoMap), ),
    center=(lat, lon)
)
zoom_slider = IntSlider(description='Zoom', min=3, max=17, value=zoom_start)
jslink((zoom_slider, 'value'), (m, 'zoom'))

In [6]:
with open('/home/deeplearning/detect_cwd/nordfjella.json') as f:
    geo_json_data = json.load(f)
layer_allzone = ipyl.GeoJSON(data=geo_json_data)
m.add_layer(layer_allzone)

In [7]:
with open('/home/deeplearning/detect_cwd/zone_1.json') as f:
    zone1_data = json.load(f)

In [8]:
with open('/home/deeplearning/detect_cwd/zone_2.json') as f:
    zone2_data = json.load(f)

In [9]:
layer_zone1 = ipyl.GeoJSON(data=zone1_data)
m.add_layer(layer_zone1)
layer_zone2 = ipyl.GeoJSON(data=zone2_data)
m.add_layer(layer_zone2)

In [10]:
polygon = ipyl.Polygon(
    locations=[(60.68753821944088,7.752502404083099), (60.75302523120072,7.809503970433292), 
               (60.80428536387097,7.559768404156764), (60.73925825494969,7.509379087922763)],
    color="red",
    fill_color="red"
)

m.add_layer(polygon);


In [18]:
# Black if healthy and red if CWD
markers = ()
ninside=0
for lon,lat in zip(reindeers['lon'],reindeers['lat']):
    point = Point(lon,lat)
    outside=True
    for s in zone1_data['features']:
        polygon = shape(s['geometry'])
        if point.within(polygon):
            print("INSIDE: Raise alarm")
            outside=False
            ninside = ninside+1
            
    if (outside):
        print("OUTSIDE")  
        markers= markers + (ipyl.Marker(location=(lat,lon)),)

if ninside>0:
    alarm = HTML()
    alarm.value = "ALARM: " + str(ninside) + " reindeers in the RED ZONE"

    # Popup with a given location on the map:
    popup = ipyl.Popup(
        location=(60.77, 7.65),
        child=alarm,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    m.add_layer(popup)
    marker.on('mouseover', function(e) {
  //open popup;
  var popup = L.popup()
   .setLatLng(e.latlng) 
   .setContent('Popup')
   .openOn(map);
});
marker_cluster = ipyl.MarkerCluster(
        markers=markers
)            
m.add_layer(marker_cluster)

INSIDE: Raise alarm
INSIDE: Raise alarm
INSIDE: Raise alarm
OUTSIDE
OUTSIDE
OUTSIDE
OUTSIDE
OUTSIDE
OUTSIDE
OUTSIDE


# Add a Marker if you see a Reindeer

In [16]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [17]:
zoom_slider

IntSlider(value=6, description='Zoom', max=17, min=3)

In [ ]:
#Add a marker when you see a reindeer 
dc = ipyl.DrawControl(circle={'shapeOptions': {'color': 'magenta'}}, circleMarker={}, polyline={}, polygon={})

def handle_draw(self, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)